In [3]:
# !pip install tensorflow_hub
# !pip install tensorflow
# !pip install tensorflow_text
# !pip install scikit-learn

In [22]:
# !pip install tensorflow_text==2.10.*

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 KB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 109.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: tensorboard
    Found existing inst

In [1]:
!pip install --upgrade tensorflow==2.10.*

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import shutil
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [4]:
 tf.__version__

'2.10.1'

#### loading movie dataset from standford

In [7]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz',
                                  url,
                                  extract=True,
                                  cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
train_dir = os.path.join(dataset_dir, 'train')
# remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

84125825/84125825 [==============================] - 8s 0us/step


In [5]:
# autotune helps in increasing the performance by load the data efficiently in the cpu
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
                                                        'aclImdb/train',
                                                        batch_size=batch_size,
                                                        validation_split=0.2,
                                                        subset='training',
                                                        seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.text_dataset_from_directory(
                                                    'aclImdb/train',
                                                    batch_size=batch_size,
                                                    validation_split=0.2,
                                                    subset='validation',
                                                    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.utils.text_dataset_from_directory(
                                                    'aclImdb/test',
                                                    batch_size=batch_size)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [6]:
for text_batch, label_batch in train_ds.take(1):
    print(f'Review: {text_batch.numpy()[0]}')
    label = label_batch.numpy()[0]
    print(f'Label : {label} ({class_names[label]})')
    break

Review: b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label : 0 (neg)


#### there are many bert models available, choosing based on use case<br> the bert model is a trained model by google

#### bert requires two steps<br>one is preprocessing the text where it converts text to number<br> another is encoder where the converted text is processed

In [7]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1")

In [8]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']
get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[ 0.98969334,  0.9486513 ,  0.1471439 , ...,  0.2550564 ,
        -0.60446906, -0.06834906],
       [ 0.9281106 ,  0.9941617 , -0.1362703 , ...,  0.5559316 ,
         0.24711996, -0.6370716 ]], dtype=float32)>

In [9]:
e = get_sentence_embeding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

print(cosine_similarity([e[0]],[e[1]]))
print(cosine_similarity([e[0]],[e[3]]))

[[0.8987519]]
[[0.8292004]]


##### Building bert model using functional model

In [10]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [12]:
# METRICS = [
#       tf.keras.metrics.BinaryAccuracy(name='accuracy'),
#       tf.keras.metrics.Precision(name='precision'),
#       tf.keras.metrics.Recall(name='recall')
# ]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=tf.keras.metrics.BinaryAccuracy(name='accuracy'))

In [13]:
model.fit(x=train_ds,validation_data=val_ds,epochs=5)

Epoch 1/5
625/625 [==============================] - 111s 171ms/step - loss: 0.5866 - accuracy: 0.6902 - val_loss: 0.5379 - val_accuracy: 0.7324
Epoch 2/5
625/625 [==============================] - 99s 158ms/step - loss: 0.5435 - accuracy: 0.7257 - val_loss: 0.5231 - val_accuracy: 0.7400
Epoch 3/5
625/625 [==============================] - 101s 162ms/step - loss: 0.5328 - accuracy: 0.7339 - val_loss: 0.5172 - val_accuracy: 0.7468
Epoch 4/5
625/625 [==============================] - 99s 158ms/step - loss: 0.5283 - accuracy: 0.7387 - val_loss: 0.5154 - val_accuracy: 0.7490
Epoch 5/5
625/625 [==============================] - 105s 168ms/step - loss: 0.5247 - accuracy: 0.7364 - val_loss: 0.5132 - val_accuracy: 0.7498


In [14]:
loss, accuracy = model.evaluate(test_ds)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

782/782 [==============================] - 116s 148ms/step - loss: 0.5171 - accuracy: 0.7470
Loss: 0.51707524061203
Accuracy: 0.746999979019165


#### testing with custom data

In [15]:
reviews = [
    'this is such an amazing movie!',
    'The movie was Terible!',
    'The movie was meh.',
    'The movie was okish.',
    'The movie was nice...'
]
predicted=model.predict(reviews)
result = np.where(predicted > 0.5, 'Good', 'Bad')
print(result)

1/1 [==============================] - 1s 590ms/step
[['Good']
 ['Bad']
 ['Bad']
 ['Good']
 ['Good']]


In [16]:
save_option = tf.saved_model.SaveOptions(experimental_io_device="/job:localhost")

In [17]:
model.save('model.h5',options=save_option)